
---

### 🦁 Projeto Aurum: Documentação do Motor de Backtest

#### 1. Visão Geral

O **Projeto Aurum** é um sistema de *Quantitative Factor Investing* (Investimento Quantitativo Fatorial) focado no mercado acionário brasileiro (B3). O objetivo do motor de backtest é simular o comportamento histórico de uma estratégia baseada no **Aurum Quality Score**, garantindo integridade de dados e execução livre de viéses.

---

#### 2. Fluxo de Arquitetura (Pipeline)

O processo de simulação segue um fluxo linear rigoroso para transformar dados brutos em decisões de investimento.

```mermaid
graph TD
    A[Dados Brutos<br/>CVM + Yahoo Finance] -->|ETL & Limpeza| B(Camada Silver<br/>Tratamento de Dados)
    B -->|Cálculo de Fatores| C(Feature Engineering<br/>ROIC, Margens, Dívida)
    C -->|Ranking & Pesos| D{Aurum Score<br/>0 a 100}
    
    subgraph "Sanity Check (Filtro de Segurança)"
    D --> E[Filtro de Anomalias]
    E -->|Banir Tickers| E1[Remove Variações >100%]
    end
    
    E -->|Dados Limpos| F[Feature Store<br/>.parquet]
    
    subgraph "Motor Backtrader"
    F --> G[Cerebro Engine]
    G --> H{Rebalanceamento Mensal}
    H -->|Compra| I[Top 5 Melhores Scores]
    H -->|Venda| J[Ativos que saíram do Ranking]
    end
    
    I & J --> K[Relatório de Performance]

```

---

### 3. Etapas do Processo

##### 🟢 Fase 1: Ingestão e Integridade (Sanity Check)

Antes de qualquer negociação, os dados passam por um rigoroso processo de saneamento para evitar "lucros fantasmas" gerados por erros de base de dados.

* **Fonte de Dados:** Demonstrações Financeiras padronizadas (CVM) e cotações ajustadas (Yahoo Finance/B3).
* **Lag Temporal (Defesa contra Look-Ahead Bias):** Aplica-se um atraso de **3 meses** entre a data do balanço e a data de negociação. Isso garante que o algoritmo só "veja" dados que o público realmente já conhecia na época.
* **Filtro de Sanidade (Novo):** Um algoritmo varre o histórico de preços em busca de variações mensais irreais (ex: >100% ou <-90% em 30 dias). Tickers com histórico corrompido (erros de split/bonificação) são **banidos** da simulação.

#### 🟡 Fase 2: O Motor de Inteligência (Aurum Score)

Cada ativo recebe uma nota mensal baseada em três pilares fundamentais:

1. **Eficiência:** Margens Operacionais e Líquidas.
2. **Rentabilidade:** ROE (Retorno sobre Patrimônio) e ROIC (Retorno sobre Capital Investido).
3. **Solvência:** Níveis de endividamento e liquidez.

O resultado é um Ranking decrescente. As ações com maiores notas são consideradas "Superiores" (High Quality).

#### 🔴 Fase 3: A Estratégia de Execução (Setup)

O motor `Backtrader` executa a estratégia seguindo regras fixas, sem intervenção humana:

* **Universo:** IBRX-100 (As 100 ações mais líquidas da bolsa).
* **Periodicidade:** Mensal.
* **Seleção:** Compra das **Top 5** ações do Ranking Aurum.
* **Alocação:** *Equal Weight* (Pesos Iguais). 20% do capital para cada ativo.
* **Regra de Saída:** Se uma ação cair para fora do Top 5 na virada do mês, ela é vendida integralmente e substituída pela nova candidata.
* **Custos:** Simulação de corretagem (0.05%) por ordem.

---

### 4. Resultado do Backtest (Proof of Concept)

A simulação realizada com dados higienizados apresentou os seguintes resultados:

| Métrica | Resultado |
| --- | --- |
| **Capital Inicial** | R$ 100.000,00 |
| **Capital Final** | **R$ 133.693,83** |
| **Retorno Total** | **+33,69%** |
| **Janela Operacional** | Set/2022 a Dez/2023 (~15 meses de operação ativa) |
| **Ativos Vencedores** | PETR4, RECV3, CURY3, WEGE3 |

### 🧠 Análise de Comportamento (Rotação Setorial)

O algoritmo demonstrou capacidade de adaptação aos ciclos econômicos:

1. **Ciclo de Commodities (2022):** Forte alocação em Petróleo (`PETR4`, `RECV3`) durante o auge dos dividendos.
2. **Ciclo Doméstico (2023):** Rotação automática para Varejo e Construção (`CURY3`, `ALOS3`, `POMO4`) antecipando a queda de juros no Brasil.

---

### 5. Conclusão Técnica

O sistema Aurum provou ser robusto após a implementação das camadas de limpeza de dados. A estratégia conseguiu gerar **Alpha** (retorno acima do mercado) através de uma seleção puramente quantitativa, validando a tese de que *Qualidade e Fundamentos* são preditores eficazes de retorno no médio prazo.

In [ ]:
import backtrader as bt
import pandas as pd
import datetime
import logging

logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

class AurumData(bt.feeds.PandasData):
    """
    1. Resolve a falta de Open/High/Low usando Adj Close.
    2. Adiciona Volatilidade (disponível no seu dataset).
    """
    lines = ('aurum_score', 'roe', 'roic', 'volatility',)

    params = (
        ('datetime', None),
        
        ('open', 'Adj Close'),  
        ('high', 'Adj Close'),
        ('low', 'Adj Close'),
        ('close', 'Adj Close'), 
        
        ('volume', -1), 
        ('openinterest', None),
        
        ('aurum_score', 'aurum_quality_score'),
        ('roe', 'ROE'),
        ('roic', 'ROIC'),
        ('volatility', 'VOLATILIDADE'),
    )

In [ ]:
class AurumRankingStrategy(bt.Strategy):
    params = (
        ('top_n', 10),           
        ('rebalance_days', 1),  
        ('reserve_cash', 0.05), 
    )

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} | {txt}')

    def __init__(self):
        self.inds = {}
        for d in self.datas:
            self.inds[d] = {
                'score': d.aurum_score,
                'name': d._name
            }
        self.timer_count = 0

    def next(self):
        self.timer_count += 1
        if self.timer_count < self.params.rebalance_days:
            return
        
        self.timer_count = 0
        
        self.log(f'--- INICIANDO REBALANCEAMENTO (Cash: {self.broker.get_cash():.2f}) ---')

        candidates = []
        for d in self.datas:
            if len(d) > 0 and d.close[0] > 0 and d.aurum_score[0] > 0:
                candidates.append((d, d.aurum_score[0]))

        candidates.sort(key=lambda x: x[1], reverse=True)

        top_stocks = [x[0] for x in candidates[:self.params.top_n]]
        top_names = [d._name for d in top_stocks]
        
        self.log(f'TOP {self.params.top_n} ATUAIS: {top_names}')

        
        for d in self.datas:
            if self.getposition(d).size > 0:
                if d not in top_stocks:
                    self.log(f'VENDENDO {d._name} (Saiu do Ranking)')
                    self.close(d) 

        target_pct = (1.0 - self.params.reserve_cash) / self.params.top_n
        
        for d in top_stocks:
            self.order_target_percent(d, target=target_pct)

In [ ]:
def filtrar_tickers_problematicos(df, limite_alta=1.0, limite_baixa=-0.8):
    """
    Remove tickers que tenham variações mensais absurdas (>100% ou <-80%).
    Isso elimina erros de split/agrupamento do Yahoo Finance.
    """
    print("\n🧹 Iniciando Filtro de Sanidade dos Dados...")
    tickers_originais = df['ticker'].unique()
    
    df['retorno'] = df.groupby('ticker')['Adj Close'].pct_change()
    
    bad_tickers = df[
        (df['retorno'] > limite_alta) | 
        (df['retorno'] < limite_baixa)
    ]['ticker'].unique()
    
    if len(bad_tickers) > 0:
        print(f"🚫 BANINDO {len(bad_tickers)} tickers com dados corrompidos:")
        print(f"   -> {list(bad_tickers)}")
        
        df_limpo = df[~df['ticker'].isin(bad_tickers)].copy()
        print(f"✅ Tickers restantes: {len(df_limpo['ticker'].unique())} de {len(tickers_originais)}")
        return df_limpo
    else:
        print("✅ Nenhum erro grave detectado nos dados.")
        return df

In [18]:
def run_strategy():
    print("--- 🦁 INICIANDO BACKTEST AURUM (COM FILTRO DE DADOS) ---")
    cerebro = bt.Cerebro()

    path_data = "../data/aurum_master_features.parquet"
    try:
        df_master = pd.read_parquet(path_data)
        df_master['date'] = pd.to_datetime(df_master['date'])
        df_master = df_master.sort_values('date')
    except Exception as e:
        print(f"Erro: {e}"); return

    df_master = filtrar_tickers_problematicos(df_master)

    start_date = '2023-01-01'
    end_date = '2025-12-31'
    mask = (df_master['date'] >= start_date) & (df_master['date'] <= end_date)
    df_filtered = df_master.loc[mask]
    
    tickers = df_filtered['ticker'].unique()
    print(f"\nCarregando {len(tickers)} feeds para o Backtrader...")

    for ticker in tickers:
        df_ticker = df_filtered[df_filtered['ticker'] == ticker].copy()
        df_ticker = df_ticker.set_index('date').sort_index()

        if len(df_ticker) < 6: continue 

        data_feed = AurumData(
            dataname=df_ticker, name=ticker,
            fromdate=pd.to_datetime(start_date), todate=pd.to_datetime(end_date)
        )
        cerebro.adddata(data_feed)

    cerebro.addstrategy(AurumRankingStrategy, top_n=5)
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.0005)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

    print(f'\n💰 Saldo Inicial: R$ {cerebro.broker.getvalue():,.2f}')
    results = cerebro.run()
    strat = results[0]
    
    final_val = cerebro.broker.getvalue()
    roi = ((final_val - 100000) / 100000) * 100

    print(f'💰 Saldo Final:   R$ {cerebro.broker.getvalue():,.2f}')
    print(f'📈 Retorno Total: {roi:.2f}%')
    try:
        print(f'📊 Sharpe Ratio:  {strat.analyzers.sharpe.get_analysis()["sharperatio"]:.3f}')
        print(f'📉 Max Drawdown:  {strat.analyzers.drawdown.get_analysis()["max"]["drawdown"]:.2f}%')
    except: pass

if __name__ == '__main__':
    run_strategy()

--- 🦁 INICIANDO BACKTEST AURUM (COM FILTRO DE DADOS) ---

🧹 Iniciando Filtro de Sanidade dos Dados...
🚫 BANINDO 19 tickers com dados corrompidos:
   -> ['COGN3.SA', 'PRIO3.SA', 'GOAU4.SA', 'USIM5.SA', 'FLRY3.SA', 'PCAR3.SA', 'MGLU3.SA', 'AZZA3.SA', 'SLCE3.SA', 'PSSA3.SA', 'CPLE3.SA', 'IRBR3.SA', 'LREN3.SA', 'CSAN3.SA', 'CPLE5.SA', 'UGPA3.SA', 'TOTS3.SA', 'CEAB3.SA', 'TEND3.SA']
✅ Tickers restantes: 75 de 94

Carregando 75 feeds para o Backtrader...

💰 Saldo Inicial: R$ 100,000.00
2025-06-30 | --- INICIANDO REBALANCEAMENTO (Cash: 100000.00) ---
2025-06-30 | TOP 5 ATUAIS: ['CURY3.SA', 'IGTI11.SA', 'VIVA3.SA', 'ALOS3.SA', 'VALE3.SA']
2025-07-01 | --- INICIANDO REBALANCEAMENTO (Cash: 5024.94) ---
2025-07-01 | TOP 5 ATUAIS: ['CURY3.SA', 'IGTI11.SA', 'VIVA3.SA', 'ALOS3.SA', 'VALE3.SA']
2025-07-31 | --- INICIANDO REBALANCEAMENTO (Cash: 5024.94) ---
2025-07-31 | TOP 5 ATUAIS: ['CURY3.SA', 'IGTI11.SA', 'VIVA3.SA', 'ALOS3.SA', 'VALE3.SA']
2025-08-01 | --- INICIANDO REBALANCEAMENTO (Cash: 4807.88

In [17]:
import pandas as pd
import backtrader as bt
import os
import sys

# --- CONFIGURAÇÕES ---
FILE_PATH = "../data/aurum_master_features.parquet"

def debug_data_loading():
    print("🔴 1. INICIANDO DIAGNÓSTICO DE DADOS...")
    
    # 1. Checagem Física do Arquivo
    if not os.path.exists(FILE_PATH):
        print(f"❌ ERRO CRÍTICO: O arquivo não existe no caminho: {os.path.abspath(FILE_PATH)}")
        print("   Verifique se a pasta '../data' está correta em relação a onde você está rodando este script.")
        return None
    print(f"✅ Arquivo encontrado em: {os.path.abspath(FILE_PATH)}")

    # 2. Carregamento com Pandas
    try:
        df = pd.read_parquet(FILE_PATH)
        print(f"✅ Arquivo Parquet carregado. Shape: {df.shape}")
    except Exception as e:
        print(f"❌ ERRO AO LER PARQUET: {e}")
        return None

    # 3. Validação de Colunas
    required_cols = ['date', 'ticker', 'Adj Close', 'aurum_quality_score']
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        print(f"❌ COLUNAS FALTANDO: {missing}")
        print(f"   Colunas disponíveis: {df.columns.tolist()}")
        return None
    print("✅ Colunas essenciais presentes.")

    # 4. Validação de Tipos de Dados (O erro mais comum no Backtrader!)
    print("\n🔍 Verificando Tipos de Dados:")
    print(df.dtypes)
    
    # Check Data
    if not pd.api.types.is_datetime64_any_dtype(df['date']):
        print("⚠️ AVISO: Coluna 'date' não é datetime. Tentando converter...")
        df['date'] = pd.to_datetime(df['date'])
    
    # Check Float
    if not pd.api.types.is_float_dtype(df['Adj Close']):
        print("❌ ERRO: 'Adj Close' não é float (número decimal). O Backtrader não vai ler!")
        return None

    # 5. Filtragem de Teste
    print("\n🔍 Testando Filtro de Datas:")
    df = df.sort_values('date')
    start_date = '2020-01-01'
    mask = df['date'] >= start_date
    df_filtered = df.loc[mask]
    
    unique_tickers = df_filtered['ticker'].unique()
    print(f"   Tickers encontrados após 2020: {len(unique_tickers)}")
    
    if len(unique_tickers) == 0:
        print("❌ ERRO: O filtro de data retornou 0 tickers. Verifique as datas no seu Parquet.")
        print(f"   Data Mínima no arquivo: {df['date'].min()}")
        print(f"   Data Máxima no arquivo: {df['date'].max()}")
        return None

    # Pega o primeiro ticker que tiver dados suficientes
    test_ticker = unique_tickers[0]
    df_one = df_filtered[df_filtered['ticker'] == test_ticker].copy()
    print(f"✅ Selecionado para teste do Backtrader: {test_ticker} ({len(df_one)} linhas)")

    return df_one, test_ticker

def test_backtrader_engine(df, ticker_name):
    print("\n🔴 2. TESTANDO MOTOR BACKTRADER (SIMPLES)...")
    
    # Prepara o DF para o Backtrader (Index deve ser data)
    df = df.set_index('date').sort_index()
    
    # Preenchimento básico
    for col in ['Open', 'High', 'Low']:
        if col not in df.columns: df[col] = df['Adj Close']
    if 'Volume' not in df.columns: df['Volume'] = 0

    cerebro = bt.Cerebro()

    # Data Feed Genérico (Sem as colunas personalizadas por enquanto, só pra ver se roda)
    data = bt.feeds.PandasData(
        dataname=df,
        name=ticker_name,
        open='Open', high='High', low='Low', close='Adj Close', volume='Volume'
    )
    
    cerebro.adddata(data)
    
    print("   Iniciando Cerebro.run()...")
    try:
        cerebro.run()
        print("✅ SUCESSO! O Backtrader processou os dados sem erro.")
    except Exception as e:
        print(f"❌ O BACKTRADER FALHOU: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    result = debug_data_loading()
    if result is not None:
        df_test, ticker = result
        test_backtrader_engine(df_test, ticker)
    else:
        print("\n❌ ABORTADO: Corrija os erros de dados acima antes de tentar o Backtrader.")

🔴 1. INICIANDO DIAGNÓSTICO DE DADOS...
✅ Arquivo encontrado em: c:\Users\kaike\projeto_aurum\aurum\data\aurum_master_features.parquet
✅ Arquivo Parquet carregado. Shape: (18879, 55)
✅ Colunas essenciais presentes.

🔍 Verificando Tipos de Dados:
date                                         datetime64[ns]
ticker                                               object
Adj Close                                           float64
CNPJ_CIA                                             object
DENOM_CIA                                            object
date_balanco                                 datetime64[ns]
Custo dos Bens e/ou Serviços Vendidos               float64
EBIT                                                float64
EBT                                                 float64
Lucro Bruto                                         float64
Lucro Líquido Consolidado                           float64
Receita Líquida                                     float64
Ativo Circulante                   